In [ ]:
import os
import sys
import math
import h5py
import argparse
import numpy as np
from collections import Counter
import xml.etree.ElementTree as et 

# Keras imports
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils.vis_utils import model_to_dot
from tensorflow.keras.layers import Input, Dense, Dropout, AlphaDropout, BatchNormalization, Activation

import matplotlib
matplotlib.use('Agg')  # Need this to write images from the GSA servers.  Order matters:
import matplotlib.pyplot as plt  # First import matplotlib, then use Agg, then import plt

# IPython imports
from IPython.display import Image

# ml4h Imports
from ml4h.plots import plot_ecg
from ml4h.arguments import parse_args
from ml4h.tensor_writer_ukbb import write_tensors
from ml4h.recipes import train_multimodal_multitask
from ml4h.models import make_multimodal_multitask_model
from ml4h.tensor_generators import TensorGenerator

In [ ]:
sys.argv = ['train', 
            '--tensors', '/mnt/disks/ecg-text3/2019-04-26/', 
            '--input_tensors', 'ecg_rest', 
            '--output_tensors', 'ecg_rhythm', 'ecg_normal', 'p-axis', 'p-duration', 
                'p-offset', 'p-onset', 'pp-interval', 'pq-interval', 'q-offset', 'q-onset', 
                'qrs-num', 'qrs-duration', 'r-axis', 'ventricular-rate',
            '--batch_size', '32', 
            '--epochs', '1',  
            '--learning_rate', '0.0001',
            '--training_steps', '20',
            '--inspect_model',
            '--id', 'ecg_regresser']
args = parse_args()
model = make_multimodal_multitask_model(**args.__dict__)


In [ ]:
COLOR_ARRAY = ['red', 'indigo', 'cyan', 'pink', 'purple', 'blue', 'chartreuse', 'darkseagreen', 'green', 'salmon',
               'magenta', 'aquamarine', 'gold', 'coral', 'tomato', 'grey', 'black', 'maroon', 'hotpink', 'steelblue',
               'orange']
def plot_ecg(data, label, prefix='./figures/'):
    lw = 1
    matplotlib.rcParams.update({'font.size': 7})

    rows = int(math.ceil(data.shape[-1]))
    cols = 1
    fig, axes = plt.subplots(rows, cols, figsize=(14, 14), sharex=True)
    for i in range(data.shape[-1]):
        color = np.random.choice(COLOR_ARRAY)
        ax = plt.subplot(rows, cols, i + 1)
        ax.plot(data[:, i], color=color, lw=lw)
    #plt.tight_layout()
    plt.title(label)
    figure_path = os.path.join(prefix, label + '_ecg.png')
    if not os.path.exists(os.path.dirname(figure_path)):
        os.makedirs(os.path.dirname(figure_path))
    plt.savefig(figure_path)
    print(f"Saved ECG plot at: {figure_path}")

In [ ]:
xdir = '/mnt/disks/ecg-text3/2019-04-26/'
paths = [xdir+'1238558.hd5', xdir+'1240625.hd5', xdir+'1258475.hd5', xdir+'1286494.hd5', xdir+'1381627.hd5']
generator = TensorGenerator(3, args.tensor_maps_in, args.tensor_maps_out, paths, None, True)
data, labels, _, hd5s = next(generator)
for i,hd5p in enumerate(hd5s):
    with h5py.File(hd5s[i], 'r') as hd5:
        print(str(hd5['ecg_rest_text'][0]).strip())
    plot_ecg(data['input_strip_ecg_rest'][i], os.path.basename(hd5s[i]))

In [ ]:
Image('./figures/1238558.hd5_ecg.png')

In [ ]:
Image('./figures/1240625.hd5_ecg.png')

In [ ]:
Image('./figures/1258475.hd5_ecg.png')

In [ ]:
paths = [xdir+'1719189.hd5', xdir+'1723645.hd5', xdir+'1767593.hd5', xdir+'1769144.hd5', xdir+'1770473.hd5']
generator = TensorGenerator(3, args.tensor_maps_in, args.tensor_maps_out, paths, None, True)
data, labels, _, hd5s = next(generator)
for i,hd5p in enumerate(hd5s):
    with h5py.File(hd5s[i], 'r') as hd5:
        print(str(hd5['ecg_rest_text'][0]).strip())
    plot_ecg(data['input_strip_ecg_rest'][i], os.path.basename(hd5s[i]))

In [ ]:
Image('./figures/1719189.hd5_ecg.png')

In [ ]:
Image('./figures/1723645.hd5_ecg.png')

In [ ]:
Image('./figures/1767593.hd5_ecg.png')

In [ ]:
h5 = '/mnt/disks/data/generated/tensors/test/2019-03-21/1005522.hd5'
with h5py.File(h5, 'r') as hd5:
    print(list(hd5['ecg_bike'].keys()))
    print(list(hd5['ecg_bike']['full_0'].shape))
    plot_ecg(hd5['ecg_bike']['full_0'][100000:102000, :], 'full')
    plot_ecg(hd5['ecg_bike']['median_0'], 'median_0')
    plot_ecg(hd5['ecg_bike']['strip_0'], 'strip_0')

In [ ]:
Image('./figures/full_ecg.png')

In [ ]:
Image('./figures/median_0_ecg.png')

In [ ]:
Image('./figures/strip_0_ecg.png')

In [ ]:
h5 = '/mnt/disks/data/generated/tensors/test/2019-03-21/1005234.hd5'
with h5py.File(h5, 'r') as hd5:
    print(list(hd5['ecg_bike'].keys()))
    print(list(hd5['ecg_bike']['full_0'].shape))
    plot_ecg(hd5['ecg_bike']['full_0'][1000:2000, :], 'full01')
    plot_ecg(hd5['ecg_bike']['median_0'], 'median_01')
    plot_ecg(hd5['ecg_bike']['strip_0'], 'strip_01')

In [ ]:
Image('./figures/full01_ecg.png')

In [ ]:
Image('./figures/median_01_ecg.png')

In [ ]:
Image('./figures/strip_01_ecg.png')